In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
ranking = pd.read_csv(os.path.join("..","raw_data","support","2018-university-ranking-2.csv"))
people = pd.read_csv(os.path.join("..","raw_data","people.csv"))
relationships = pd.read_csv(os.path.join("..","raw_data","relationships.csv"))
comps =pd.read_csv(os.path.join("..","raw_data","companies.csv"))

In [3]:
degrees = pd.read_csv(os.path.join("..","raw_data","degrees.csv"))

In [5]:
test = pd.DataFrame(degrees.institution.str.contains('university of sydney',case = False))
lista = list(test[test.institution==True].index)
add = []
for i in lista:
    if degrees.institution[i] not in add:
        add.append(degrees.institution[i])    

print (add)

['University of Sydney', 'University of Sydney/ TU Berlin,', 'The University of Sydney', 'University of Sydney - Engineering', 'University of Innsbruck / University of Sydney', 'University of Sydney, Australia', 'University of Sydney Business School', 'University of Sydney Law School']


In [6]:
def degree_standardisation(x):
    if "harvard" in x:
        return "harvard university"
    if "stanford" in x:
        return "stanford university"
    if "massachusetts institute of technology (mit)" in x:
        return "massachusetts institute of technology (mit)"
    if "massachusetts institute of technology" in x:
        return "massachusetts institute of technology (mit)"
    if "mit " in x:
        return "massachusetts institute of technology (mit)"
    if " mit " in x:
        return "massachusetts institute of technology (mit)"
    if "(mit)" in x:
        return "massachusetts institute of technology (mit)"
    if "california institue of technology" in x:
        return "california institue of technology (caltech)"
    if "caltech" in x:
        return "california institue of technology (caltech)"
    if "cambridge" in x:
        return "university of cambridge"
    if "oxford" in x: 
        return "university of oxford"
    if "university college london" in x:
        return "(ucl) university college london"
    if "imperial college" in x:
        return "imperial college london"
    if "university of chicago" in x:
        return "university of chicago"
    if "chicago university" in x:
        return "university of chicago"
    if "eth zurich" in x:
        return "eth zurich (swiss federal institute of technology)"
    if "nanyang" in x:
        return "nanyang technological university (NTU)"
    if "epfl" in x: 
        return "ecole polytechnique fédérale de lausanne (epfl)"
    if all(c in x for c in ["ecole polytechnique","lausanne"]):
        return "ecole polytechnique fédérale de lausanne (epfl)"
    if 'princeton' in x:
        return "princeton university"
    if "cornell" in x:
        return "cornell university"
    if all(c in x for c in ["national university","singapore"]):
        return 'national university of singapore (nus)'
    if "yale" in x:
        return "yale university"
    if "john hopkins" in x:
        return "joh hopkins university"
    if "columbia" in x:
        return "columbia university"
    if "pennsylvania" in x:
        return "university of pennsylvania"
    if "upenn" in x:
        return "university of pennsylvania"
    if "duke university" in x:
        return "duke university"
    if "michigan" in x:
        return "michigan"
    if "australian national university" in x:
        return "australian national unversity (anu)"
    if all (c in x for c in ["king","college"]):
        return "king's college london (kcl)"
    if all (c in x for c in ["university","edinburgh"]):
        return "university of edinburgh"
    if "tsinghua" in x:
        return "tsinghua university"
    if "hkust" in x: 
        return "the hong kong university of science and technology (hkust)"
    if "hku" in x:
        return "university of hong kong (hku)"
    if all (c in x for c in ["hong kong", "technology", "science"]):
        return "the hong kong university of science and technology (hkust)"
    if "hong kong" in x:
        return "university of hong kong (hku)"
    if "berkeley" in x:
        return 'university of california, berkeley (ucb)'
    if "northwestern" in x:
        return "northwestern university"
    if "tokyo" in x:
        return 'the university of tokyo'
    if "toronto" in x:
        return "university of toronto"
    if "mcgill" in x:
        return "mcgill university"
    if "ucla" in x:
        return "university of california, los angeles (ucla)"
    if all (c in x for c in ["california", "los angeles", "university"]):
        return "university of california, los angeles (ucla)"
    if "manchester" in x:
        return "the university of manchester"
    if "london school of economics" in x:
        return "london school of economics and political science (lse)"
    if "kyoto" in x:
        return "kyoto university"
    if "seoul" in x:
        return "seoul national university (snu)"
    if "peking university" in x:
        return "peking university"
    if "ucsd" in x:
        return "university of california, san diego (ucsd)"
    if all (c in x for c in ["california", "san diego", "university"]):
        return "university of california, san diego (ucsd)"
    if "fudan" in x:
        return "fudan university"
    if "kaist" in x:
        return "kaist - korea advanced institute of science and technology"
    if "korea advanced institute of science and technology" in x:
        return "kaist - korea advanced institute of science and technology"
    if all (c in x for c in ["university","melbourne"]):
        return "the university of melbourne"
    if "ecole normale sup" in x:
        return "ecole normale supérieure, paris, (ens paris)"
    if "école normale sup" in x:
        return "ecole normale supérieure, paris, (ens paris)"
    if "ens paris" in x:
        return "ecole normale supérieure, paris, (ens paris)"
    if all (c in x for c in ["university","bristol"]):
        return "university of bristol"
    if "new south wales" in x:
        return 'the university of new south wales (unsw)'
    
    if all (c in x for c in ["Chinese University of Hong Kong","CUHK"]):
        return 'the chinese university of hong kong (cuhk)'
    
    if 'UQ' in x:
        return 'the university of queensland (uq)'

    if "city university" == x:
        return 'city university of hong kong'
    
    if "City University of Hong Kong" == x:
        return 'city university of hong kong'
    
    if "University of Sydney" in x:
        return 'the university of sydney'

    else:
        return 'no top'

In [7]:

def get_people_top(degrees, people):
    ''' get a new column is_top50? for each employee in the people table'''
    degrees.institution = degrees.institution.fillna('No info')
    degrees['degree_standardization'] = degrees.institution.map(lambda x: degree_standardisation(x.lower()))
    degrees['is_top50'] = degrees.degree_standardization.map(lambda x: 0 if x=='no top' else 1)
    is_top_df = pd.DataFrame(degrees.groupby('object_id')['is_top50'].max()).reset_index().rename(columns = {'object_id': 'person_object_id'})
    merge = people.merge(is_top_df, how = 'left', on ='person_object_id')
    return merge

In [8]:
def count_top_employees(relationships, degrees, people, comps):
    ''' get the companies table with the total number of top50 employees'''
    
    get_people = get_people_top(degrees, people)
    
    merge = relationships.merge(get_people, how= 'left', on = 'person_object_id')
    
    top_by_comp = pd.DataFrame(merge.groupby('relationship_object_id')['is_top50'].count()).reset_index().rename(columns = {'relationship_object_id': 'id'})
    
    merge_2 = comps.merge(top_by_comp, how= 'left', on = 'id')
    
    return merge_2